In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
from keras.applications import VGG16
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, Lambda
from keras.models import Model, load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
from PIL import Image
from tqdm import tqdm, trange

In [2]:
nb_classes = 1  # number of classes
img_width, img_height = 224, 224  # change based on the shape/structure of your images
batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 50  # number of iteration the algorithm gets trained.
learn_rate = 1e-5  # sgd learning rate

## DIR ##

In [ ]:
train_dir = 'd:/data/preprocessed_dataset/train'
validation_dir = 'd:/data/preprocessed_dataset/validation'
test50_dir = 'd:/data/preprocessed_dataset/test50'
test75_dir = 'd:/data/preprocessed_dataset/test75'
test95_dir = 'd:/data/preprocessed_dataset/test95'

# AlexNet

In [ ]:
img_input = Input(shape=(img_height, img_width, 3))

x = Conv2D(96, 11, strides=4, padding='same', use_bias=False)(img_input) # 15
x = Activation('relu')(x)

x = Conv2D(256, 5, strides=1, padding='same', use_bias=False)(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8

x = Conv2D(384, 3, strides=1, padding='same', use_bias=False)(x) # 15
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8

x = Conv2D(384, 3, strides=1, padding='same', use_bias=False)(x) # 15
x = Activation('relu')(x)
x = Conv2D(256, 3, strides=1, padding='same', use_bias=False)(x)
x = Activation('relu')(x)

model_out = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x) # 8
# Add fully connected layer
x = GlobalAveragePooling2D()(model_out)
x = Dense(4096, activation=None)(x)
x = Activation('relu')(x)
x = Dense(1, activation=None)(x)
out = Activation('sigmoid')(x)

model = Model(img_input, out)
print(model.summary())
print(len(model.trainable_weights))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34848     
_________________________________________________________________
activation_1 (Activation)    (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       614400    
_________________________________________________________________
activation_2 (Activation)    (None, 56, 56, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 256)       0         
_________________________________________________________________
conv

In [ ]:
model.compile(optimizer=Adam(lr=learn_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(len(model.trainable_weights))

9


## Data generator

In [ ]:
def generator(directory, batch_size=32):
    folder =  np.sort(os.listdir(directory))
    real_img = np.asarray(glob.glob(directory + '/' + folder[0]+'/*.png'))
    real_idx = np.arange(len(real_img))
    
    while 1:
        X1 = []
        X2 = []
        y = []
        
        if (len(real_idx) < batch_size):
            real_idx = np.arange(len(real_img))
            continue
        
        for _ in range(batch_size):
            if (len(real_idx) < batch_size):
                real_idx = np.arange(len(real_img))
                break
            random1 = np.random.choice(real_idx, 1, replace=False)
            real_idx = real_idx[~np.isin(real_idx, random1)]
            random2 = np.random.choice(real_idx, 1, replace=False)
            real_idx = real_idx[~np.isin(real_idx, random2)]
            X1.append(np.asarray(Image.open(real_img[random1[0]]).convert("RGB"))/255.)
            X2.append(np.asarray(Image.open(real_img[random2[0]]).convert("RGB"))/255.)
            y.append(np.array([0.]))

        X1 = np.asarray(X1)
        X2 = np.asarray(X2)
        y = np.asarray(y)
        yield [X1, X2], y
        
def generator_res(ft_dir, directory, batch_size=1, critical_value=0.5):
    folder = np.sort(os.listdir(directory))
    ft_img = np.asarray(glob.glob(ft_dir + '/' + '0' +'/*.png'))
    ft_idx = np.arange(len(ft_img))
    random1 = np.random.choice(ft_idx, 1, replace=False)
    img = np.asarray(Image.open(ft_img[random1[0]]).convert("RGB"))/255.
    fake_img = np.asarray(glob.glob(directory + '/' + folder[1] + '/*.png'))
    fake_idx = np.arange(len(fake_img))
    real_img = np.asarray(glob.glob(directory + '/' + folder[0] + '/*.png'))
    real_idx = np.arange(len(real_img))
    while 1:
        X1 = []
        X2 = []
        y = []
        if (len(fake_idx) < batch_size):
            break
        if (len(real_idx) < batch_size):
            break
        for _ in range(batch_size):
            if np.random.uniform() > critical_value:
                if (len(fake_idx) < batch_size):
                    break
                random2 = np.random.choice(fake_idx, 1, replace=False)
                fake_idx = fake_idx[~np.isin(fake_idx, random2)]
                X1.append(img)
                X2.append(np.asarray(Image.open(fake_img[random2[0]]).convert("RGB"))/255.)
                y.append(np.array([1.]))
            else:
                if (len(real_idx) < batch_size):
                    break
                random3 = np.random.choice(real_idx, 1, replace=False)
                real_idx = real_idx[~np.isin(real_idx, random3)]
                X1.append(img)
                X2.append(np.asarray(Image.open(real_img[random3[0]]).convert("RGB"))/255.)
                y.append(np.array([0.]))
        X1 = np.asarray(X1)
        X2 = np.asarray(X2)
        y = np.asarray(y)
        yield [X1, X2], y

In [ ]:
def manDist(x):
    result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
    return result

def euclidean_distance(inputs):
    assert len(inputs) == 2, 'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v + 1e-7)), axis=1, keepdims=True))  

def contrastive_loss(y_true,y_pred):
    margin=1.4
    return K.mean((1. - y_true) * K.square(y_pred) + y_true * K.square(K.maximum(margin - y_pred, 0.)))

def siamese_acc(y_true, y_pred):
    return K.mean((K.equal(y_true, K.cast(y_pred > 0.4, K.floatx()))), axis=1)

def y_pred_prt(y_true, y_pred):
    return y_pred

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                        target_size=(img_height, img_width),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='binary')

test50_generator = test_datagen.flow_from_directory(test50_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode='binary')

test75_generator = test_datagen.flow_from_directory(test75_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode='binary')

test95_generator = test_datagen.flow_from_directory(test95_dir,
                                                  target_size=(img_height, img_width),
                                                  batch_size=32,
                                                  shuffle=False,
                                                  class_mode='binary')

Found 251702 images belonging to 2 classes.
Found 28298 images belonging to 2 classes.
Found 33086 images belonging to 2 classes.
Found 21866 images belonging to 2 classes.
Found 17480 images belonging to 2 classes.


In [ ]:
callback_list = [EarlyStopping(monitor='val_acc', patience=5),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)]

history = model.fit_generator(train_generator,
                            steps_per_epoch=100,
                            epochs=20,
                            validation_data=validation_generator,
                            validation_steps=len(validation_generator),
                            callbacks=callback_list,
                            verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
  6/100 [>.............................] - ETA: 14:37 - loss: 0.6938 - acc: 0.31 - ETA: 7:18 - loss: 0.6934 - acc: 0.5156 - ETA: 4:51 - loss: 0.6932 - acc: 0.520 - ETA: 4:01 - loss: 0.6932 - acc: 0.515 - ETA: 3:54 - loss: 0.6933 - acc: 0.493 - ETA: 3:48 - loss: 0.6931 - acc: 0.5052

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.100943). Check your callbacks.
  % delta_t_median)


  7/100 [=>............................] - ETA: 3:45 - loss: 0.6930 - acc: 0.5045

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.197887). Check your callbacks.
  % delta_t_median)


  9/100 [=>............................] - ETA: 3:42 - loss: 0.6929 - acc: 0.503 - ETA: 3:36 - loss: 0.6928 - acc: 0.5243

...

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.232868). Check your callbacks.
  % delta_t_median)


 98/100 [============================>.] - ETA: 3s - loss: 0.6494 - acc: 0.6215

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.223873). Check your callbacks.
  % delta_t_median)


 99/100 [============================>.] - ETA: 1s - loss: 0.6479 - acc: 0.6247

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.205890). Check your callbacks.
  % delta_t_median)


In [ ]:
ft_dir = 'd:/data/preprocessed_dataset/fine-tune'
train_gen = generator(ft_dir)
test50_gen = generator_res(ft_dir, test50_dir, batch_size=1, critical_value=0.5)
test75_gen = generator_res(ft_dir, test75_dir, batch_size=1, critical_value=0.75)
test95_gen = generator_res(ft_dir, test95_dir, batch_size=1, critical_value=0.95)

## Modeling

In [ ]:
test50_classes = test50_generator.classes
print("50% ", len(test50_classes[test50_classes == 1]))
test75_classes = test75_generator.classes
print("75% ", len(test75_classes[test75_classes == 1]))
test95_classes = test95_generator.classes
print("95%: ", len(test95_classes[test95_classes == 1]))

In [ ]:
model.save("d:/data/preprocessed_dataset/fake_alexnet.h5")

### Siamese Modeling

In [11]:
model = load_model("d:/data/preprocessed_dataset/alexnet.h5")
base_model = Model(img_input, out)
base_model.set_weights(model.get_weights())
for l in range(len(base_model.layers) - 2):
    base_model.layers[l].trainable = False   

im_in = Input(shape=(224, 224, 3))
x1 = base_model([im_in])

model_top = Model(inputs=[im_in], outputs=x1)
# model_top.load_weights("d:/data/preprocessed_dataset/siam_alex.h5")
model_top.summary()

left_input = Input(shape=(224, 224, 3))
right_input = Input(shape=(224, 224, 3))

h1 = model_top(left_input)
h2 = model_top(right_input)

distance = Lambda(euclidean_distance)([h1, h2])
siam_model = Model(inputs=[left_input, right_input], outputs=distance)
siam_model.compile(loss='mse', optimizer=SGD(0.001), metrics=['acc'])
siam_model.summary()
callback_list = [EarlyStopping(monitor='acc', patience=3),
                 ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2)]
output = siam_model.fit_generator(train_gen, steps_per_epoch=40, epochs=10,callbacks=callback_list)

Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
model_2 (Model)              (None, 1)                 4802593   
Total params: 4,802,593
Trainable params: 4,097
Non-trainable params: 4,798,496
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 224, 224, 3)  0                          

40/40 [==============================] - ETA: 7s - loss: 0.0904 - acc: 0.906 - ETA: 8s - loss: 0.1012 - acc: 0.875 - ETA: 8s - loss: 0.1148 - acc: 0.854 - ETA: 8s - loss: 0.1116 - acc: 0.859 - ETA: 8s - loss: 0.0978 - acc: 0.881 - ETA: 7s - loss: 0.0815 - acc: 0.901 - ETA: 7s - loss: 0.0810 - acc: 0.901 - ETA: 7s - loss: 0.0845 - acc: 0.898 - ETA: 6s - loss: 0.0891 - acc: 0.895 - ETA: 6s - loss: 0.0852 - acc: 0.900 - ETA: 6s - loss: 0.0857 - acc: 0.900 - ETA: 6s - loss: 0.0825 - acc: 0.903 - ETA: 6s - loss: 0.0799 - acc: 0.906 - ETA: 6s - loss: 0.0814 - acc: 0.904 - ETA: 5s - loss: 0.0824 - acc: 0.902 - ETA: 5s - loss: 0.0806 - acc: 0.902 - ETA: 5s - loss: 0.0820 - acc: 0.900 - ETA: 5s - loss: 0.0831 - acc: 0.899 - ETA: 4s - loss: 0.0817 - acc: 0.899 - ETA: 4s - loss: 0.0832 - acc: 0.898 - ETA: 4s - loss: 0.0821 - acc: 0.898 - ETA: 4s - loss: 0.0820 - acc: 0.899 - ETA: 4s - loss: 0.0846 - acc: 0.895 - ETA: 3s - loss: 0.0850 - acc: 0.895 - ETA: 3s - loss: 0.0826 - acc: 0.898 - ETA: 3s -

In [12]:
## siam model을 통째로 저장할 때 load err: axes doesn't match ##
''' model 설정 후, load weight '''
model_json = model_top.to_json()
with open("d:/data/preprocessed_dataset/siam_alex.json", "w") as json_file:
    json_file.write(model_json)
model_top.save_weights("d:/data/preprocessed_dataset/siam_alex.h5")

## Evaluate Model - 50

In [13]:
model_1 = load_model("d:/data/preprocessed_dataset/alexnet.h5")
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34848     
_________________________________________________________________
activation_1 (Activation)    (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       614400    
_________________________________________________________________
activation_2 (Activation)    (None, 56, 56, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 384)       884736    
__________

In [14]:
predictions50_1 = model_1.predict_generator(test50_generator, steps=len(test50_generator))
y_pred50_1 = predictions50_1.copy()
predictions50_1[predictions50_1 > 0.5] = 1
predictions50_1[predictions50_1 <= 0.5] = 0
true_classes50_1 = test50_generator.classes

fpr50_1, tpr50_1, thresholds50_1 = roc_curve(true_classes50_1, y_pred50_1, pos_label=1.)
cm50_1 = confusion_matrix(true_classes50_1, predictions50_1)
recall50_1 = cm50_1[0][0] / (cm50_1[0][0] + cm50_1[0][1])
fallout50_1 = cm50_1[1][0] / (cm50_1[1][0] + cm50_1[1][1])
eer50_1 = brentq(lambda x : 1. - x - interp1d(fpr50_1, tpr50_1)(x), 0., 1.)
thresh50_1 = interp1d(fpr50_1, thresholds50_1)(eer50_1)
test_loss50_1, test_acc50_1 = model_1.evaluate_generator(test50_generator, steps=len(test50_generator))

In [15]:
print(metrics.classification_report(true_classes50_1, predictions50_1))
print("FPR=FAR", fallout50_1)
print("FNR=FRR", 1-recall50_1)
print('test acc:', test_acc50_1)
print('test_loss:', test_loss50_1)
print('thresh:', thresh50_1)
print('eer:', eer50_1)
print(cm50_1)

              precision    recall  f1-score   support

           0       0.50      0.48      0.49     16543
           1       0.50      0.52      0.51     16543

   micro avg       0.50      0.50      0.50     33086
   macro avg       0.50      0.50      0.50     33086
weighted avg       0.50      0.50      0.50     33086

FPR=FAR 0.48231880553708517
FNR=FRR 0.5201595841141269
test acc: 0.9811098349502972
test_loss: 0.06298309940407233
thresh: 0.9337399005889906
eer: 0.5015414374659964
[[7938 8605]
 [7979 8564]]


In [16]:
score50_2 = []
answer50_2 = []
max_iter50_2 = int(20000)
j = 0
for i in tqdm(test50_gen):
    y_score50_2 = siam_model.predict_on_batch(i[0])
    score50_2.append(y_score50_2)
    answer50_2.append(i[1])
    j += 1
    
score50_2 = np.concatenate(score50_2)
answer50_2 = np.concatenate(answer50_2)

16942it [03:19, 86.55it/s]

KeyboardInterrupt: 

In [ ]:
y_hat50_2 = score50_2.copy()
y_hat50_2[y_hat50_2 >= 0.9] = 1.
y_hat50_2[y_hat50_2 < 0.9] = 0.

cm50_2 = confusion_matrix(answer50_2, y_hat50_2)
recall50_2 = cm50_2[0][0] / (cm50_2[0][0] + cm50_2[0][1])
fallout50_2 = cm50_2[1][0] / (cm50_2[1][0] + cm50_2[1][1])
fpr50_2, tpr50_2, thresholds50_2 = roc_curve(answer50_2, score50_2, pos_label=1.)
thresh = interp1d(fpr50_2, thresholds50_2)(eer50_2)
eer50_2 = brentq(lambda x : 1. - x - interp1d(fpr50_2, tpr50_2)(x), 0., 1.)

In [ ]:
print(metrics.classification_report(answer50_2, y_hat50_2))
print(confusion_matrix(answer50_2, y_hat50_2))
print("FPR=FAR", fallout50_2)
print("FNR=FRR", 1-recall50_2)
print('test_acc: ', len(y_hat50_2[np.equal(y_hat50_2, answer50_2)]) / len(y_hat50_2)
  print('thresh:', thresh50_2)
print('eer:', eer50_2)

## Evaluate Model - 75

In [ ]:
predictions75_1 = model_1.predict_generator(test75_generator, steps=len(test75_generator))
y_pred75_1 = predictions75_1.copy()
predictions75_1[predictions75_1 > 0.5] = 1
predictions75_1[predictions75_1 <= 0.5] = 0
true_classes75_1 = test75_generator.classes

fpr75_1, tpr75_1, thresholds75_1 = roc_curve(true_classes75_1, y_pred75_1, pos_label=1.)
cm75_1 = confusion_matrix(true_classes75_1, predictions75_1)
recall75_1 = cm75_1[0][0] / (cm75_1[0][0] + cm75_1[0][1])
fallout75_1 = cm75_1[1][0] / (cm75_1[1][0] + cm75_1[1][1])
eer75_1 = brentq(lambda x : 1. - x - interp1d(fpr75_1, tpr75_1)(x), 0., 1.)
thresh75_1 = interp1d(fpr75_1, thresholds75_1)(eer75_1)
test_loss75_1, test_acc75_1 = model_1.evaluate_generator(test75_generator, steps=len(test75_generator))

In [ ]:
print(metrics.classification_report(true_classes75_1, predictions75_1))
print("FPR=FAR", fallout75_1)
print("FNR=FRR", 1-recall75_1)
print('test acc:', test_acc75_1)
print('test_loss:', test_loss75_1)
print('thresh:', thresh75_1)
print('eer:', eer75_1)

In [ ]:
score75_2 = []
answer75_2 = []
max_iter75_2 = int(20000)
j = 0
for i in tqdm(test75_gen):
    y_score75_2 = siam_model.predict_on_batch(i[0])
    score75_2.append(y_score75_2)
    answer75_2.append(i[1])
    j += 1
    
score75_2 = np.concatenate(score75_2)
answer75_2 = np.concatenate(answer75_2)

In [ ]:
print(metrics.classification_report(answer75_2, y_hat75_2))
print(confusion_matrix(answer75_2, y_hat75_2))
print("FPR=FAR", fallout75_2)
print("FNR=FRR", 1-recall75_2)
print('test_acc: ', len(y_hat75_2[np.equal(y_hat75_2, answer75_2)]) / len(y_hat75_2)
  print('thresh:', thresh75_2)
print('eer:', eer75_2)

## Evaluate Model - 95

In [ ]:
predictions95_1 = model_1.predict_generator(test95_generator, steps=len(test95_generator))
y_pred95_1 = predictions95_1.copy()
predictions95_1[predictions95_1 > 0.5] = 1
predictions95_1[predictions95_1 <= 0.5] = 0
true_classes95_1 = test95_generator.classes

fpr95_1, tpr95_1, thresholds95_1 = roc_curve(true_classes95_1, y_pred95_1, pos_label=1.)
cm95_1 = confusion_matrix(true_classes95_1, predictions95_1)
recall95_1 = cm95_1[0][0] / (cm95_1[0][0] + cm95_1[0][1])
fallout95_1 = cm95_1[1][0] / (cm95_1[1][0] + cm95_1[1][1])
eer95_1 = brentq(lambda x : 1. - x - interp1d(fpr95_1, tpr95_1)(x), 0., 1.)
thresh95_1 = interp1d(fpr95_1, thresholds95_1)(eer95_1)
test_loss95_1, test_acc95_1 = model_1.evaluate_generator(test95_generator, steps=len(test95_generator))

In [ ]:
print(metrics.classification_report(true_classes95_1, predictions95_1))
print("FPR=FAR", fallout95_1)
print("FNR=FRR", 1-recall95_1)
print('test acc:', test_acc95_1)
print('test_loss:', test_loss95_1)
print('thresh:', thresh95_1)
print('eer:', eer95_1)

In [ ]:
score95_2 = []
answer95_2 = []
max_iter95_2 = int(20000)
j = 0
for i in tqdm(test95_gen):
    y_score95_2 = siam_model.predict_on_batch(i[0])
    score95_2.append(y_score95_2)
    answer95_2.append(i[1])
    j += 1
    
score95_2 = np.concatenate(score95_2)
answer95_2 = np.concatenate(answer95_2)

In [ ]:
print(metrics.classification_report(answer95_2, y_hat95_2))
print(confusion_matrix(answer95_2, y_hat95_2))
print("FPR=FAR", fallout95_2)
print("FNR=FRR", 1-recall95_2)
print('test_acc: ', len(y_hat95_2[np.equal(y_hat95_2, answer95_2)]) / len(y_hat95_2)
  print('thresh:', thresh95_2)
print('eer:', eer95_2)

## Plotting

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(fpr50_1, tpr50_1, 'b-', label="AlexNet (50%)")
plt.plot(fpr50_2, tpr50_2, 'r-', label="Siamese(Ours) (50%)")
plt.plot(fpr75_1, tpr75_1, 'b-', label="AlexNet (75%)")
plt.plot(fpr75_2, tpr75_2, 'r-', label="Siamese(Ours) (75%)")
plt.plot(fpr95_1, tpr95_1, 'b-', label="AlexNet (95%)")
plt.plot(fpr95_2, tpr95_2, 'r-', label="Siamese(Ours) (95%)")
plt.plot([0, 1], [0, 1], 'k--', label="random guess")
plt.plot([fallout], [recall], 'ro', ms=10)
plt.plot([fallout1], [recall1], 'bo', ms=10)
plt.xlabel('False Positive Rate (Fall-Out)')
plt.ylabel('True Positive Rate (Recall)')
plt.title("Best AUROC: %.3f / Model: Ours" %(roc_auc_score(answer, score)))
plt.legend(loc='lower right')
# plt.annotate("%.3f: AlexNet" %(roc_auc_score(true_classes, y_pred)), xy=(0.88, 0.85), xytext=(0.75, 0.70), arrowprops={'color':'blue'})
# plt.annotate("%.3f: Ours" %(roc_auc_score(answer, score)), xy=(0.0, 0.99), xytext=(0.15, 0.9), arrowprops={'color':'red'})
plt.show()